<a href="https://colab.research.google.com/github/karayolharun/myProjects/blob/master/Pat_Mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# INSTALLATION - DRIVE MOUNT MODULE ***********************************
!python -m spacy download en_core_web_lg
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# MAIN MODULE - REFERENCES-QUERY WORDS-QUERY PRODUCTION-LINK PRODUCTION *******************************************************
#------------------------SAVES OUTPUT TO TEXT FILE---------------------------
# URLs 
# REFERENCES
# NOUN-PHRASES
# QUERY

# pyc dosyası üretmek için *********************************
#import py_compile
#script = "/content/drive/My Drive/kodlar/kod.py"
#py_compile.compile(script)
# **********************************************************

import os
import shutil
from os import path

import time

import spacy
from spacy.matcher import Matcher

from collections import Counter

nlp = spacy.load('en_core_web_lg')


f = open('/content/drive/My Drive/MYOUTPUT.txt', 'a')
choice_text = input("Description or Claim ? : ")
if choice_text.lower()=="d":
  print("MYOUTPUT-D\nINPUT IS DESCRIPTION TEXT\n",file=f)
  print("INPUT IS DESCRIPTION TEXT\n")
else: 
  print("MYOUTPUT-C\nINPUT IS CLAIM TEXT\n",file=f)
  print("INPUT IS CLAIM TEXT\n")

f.close()

while True:
  oldest_date = input("For prior art search insert the priority or filing date in format:yyyymmdd  ")
 
  if int(oldest_date[4:6])>12 or int(oldest_date[6:])>31:
    print("Please enter a valid date in format: yyyymmdd  ")
    continue
  
  elif (oldest_date[4:6]=='00') or (oldest_date[6:]=='00'):
    print("Please enter a valid date in format: yyyymmdd  ")
    continue
      
  elif oldest_date[4:6]=='02' and oldest_date[6:]=='30':
    print("February cannot have 30 days. Please enter a valid date in format: yyyymmdd  ")
    continue
  
  elif oldest_date[4:6]=='02' and oldest_date[6:]=='31':
    print("February cannot have 31 days. Please enter a valid date in format: yyyymmdd  ")
    continue

  elif int(oldest_date[:4])%4 != 0 and oldest_date[4:6]=='02' and oldest_date[6:] == '29':
    print("This is not a leap year. Please enter a valid date in format: yyyymmdd  ")
    continue

  
  elif oldest_date.isnumeric and len(oldest_date)==8:
    if int(oldest_date[4:6])<13:
      if int(oldest_date[6:])<32:
        break  
  
  else:
    print("Please enter a valid date in format: yyyymmdd  ")
    continue

  
  


if (choice_text.lower()=="d"):
  words = input("Insert description text here:  ")
else:
  words = input("Insert claim text here:  ")



doc = nlp(words)
#-------------------------------------------------
f = open('/content/drive/My Drive/MYOUTPUT.txt', 'a')

#IPC SENTENCE BULMA İŞİ
try:
  ipc_sent = list()
  for sent in doc.sents:
    for token in sent:
      if (token.text.startswith("invent") or token.text.startswith("applicat")) and token.pos_ == 'NOUN':
        if doc[token.i+1].text.startswith("relat") or doc[token.i+2].text.startswith("relat")or doc[token.i+1].text.startswith("about") or doc[token.i+2].text.startswith("about") or doc[token.i+1].text.startswith("concern") or doc[token.i+2].text.startswith("concern"):
          print("\nIPC sentence:  ",sent,"\n") #sonra silinebilir
          ipc_sent.append(sent)
          print("ipc_sent listesi  :",sent,"\n") #sonra silinebilir
except:
  print("\n NO IPC SENTENCE FOUND \n")
  print("\n NO IPC SENTENCE FOUND \n",file=f)

#ipc_sent span listesi*
try:
  if (len(ipc_sent)>0):
    doc_ipc_sent = nlp(str(ipc_sent[0]))
    print ("doc_ipc_sent:  ",doc_ipc_sent)
  else:
    print("\n NO IPC SENTENCE FOUND \n")
    print("\n NO IPC SENTENCE FOUND \n",file =f)   

  ipc_sent_span = doc_ipc_sent
  for token in doc_ipc_sent:
    if token.text.startswith("concern") or token.text.startswith("relat") or token.text.startswith("about"):
      ipc_sent_span = doc_ipc_sent[token.i+1:]
      break


  ipc_sent_noun_list = list()
  for token in ipc_sent_span:
    if (token.pos_ == 'ADJ') or (token.pos_ == 'NOUN') or (token.tag_ == 'VBG'):
      if  token.text.endswith("s")==True and len(token.text)>3: #ex: "keys" sonundaki 's' atılır#
        ipc_sent_noun_list.append(str(token)[:-1])
      else:
        ipc_sent_noun_list.append(str(token))

        
#ipc_sent_noun_list elemanları string#

  ipc_sent_noun_set = set(ipc_sent_noun_list)

  ipc_sent_noun_unique_list = list(ipc_sent_noun_set)

  print ("\n IPC NOUN SET LIST: ",ipc_sent_noun_unique_list)
  print ("\n IPC NOUN SET LIST: ",ipc_sent_noun_unique_list, file=f)

  #https://patents.google.com/?q=(vehicle+seat)&q=(folding+mechanism)&q=lock&before=publication:20140624
  ipc_url_noun_addition_string = ""
  if len(ipc_sent_noun_unique_list)>0:
    for i in range(len(ipc_sent_noun_unique_list)):
      if i==0:
        ipc_url_noun_addition_string += "?q=("+ipc_sent_noun_unique_list[i]+")"
      else:
        ipc_url_noun_addition_string += "&q=("+ipc_sent_noun_unique_list[i]+")"
    ipc_noun_search_url = "https://patents.google.com/"+ipc_url_noun_addition_string+"&before=publication:"+oldest_date+"&clustered=true"    #IPC SEARCH URL PRODUCED
  else:
    print("\nNO IPC NOUN SEARCH URL PRODUCED\n")
    print("\nNO IPC NOUN SEARCH URL PRODUCED\n",file=f)



  #liste_chunk_ipc , ipc noun-chunk string listesi#
  liste_chunk_ipc=[]
  for chunk in ipc_sent_span.noun_chunks:
      if chunk[0].is_stop == True or chunk[0].text == "-" or chunk[0].is_punct == True or chunk[0].pos_ == 'NUM':
        if chunk.text.endswith("s"):
          liste_chunk_ipc.append(chunk[1:].text.rstrip("s"))
        else:
          liste_chunk_ipc.append(chunk[1:].text)
      else:
        if chunk.text.endswith("s"):
          liste_chunk_ipc.append(chunk.text.rstrip("s"))
        else:
          liste_chunk_ipc.append(chunk.text)
      
  ipc_chunk_set = set(liste_chunk_ipc)

  ipc_sent_chunk_unique_list = list(ipc_chunk_set)

  print ("\n IPC CHUNK SET LIST: ",ipc_sent_chunk_unique_list)
  print ("\n IPC CHUNK SET LIST: ",ipc_sent_chunk_unique_list, file=f)
    
    
  ipc_url_chunk_addition_string = ""
  if len(ipc_sent_chunk_unique_list)>0:
    for i in range(len(ipc_sent_chunk_unique_list)):
      if i==0:
        ipc_url_chunk_addition_string += "?q=("+ipc_sent_chunk_unique_list[i].replace(' ','+')+")"
      else:
        ipc_url_chunk_addition_string += "&q=("+ipc_sent_chunk_unique_list[i].replace(' ','+')+")"
    ipc_chunk_search_url = "https://patents.google.com/"+ipc_url_chunk_addition_string+"&before=publication:"+oldest_date+"&clustered=true"    #IPC SEARCH URL PRODUCED
  else:
    print("\nNO IPC CHUNK SEARCH URL PRODUCED\n")
    print("\nNO IPC CHUNK SEARCH URL PRODUCED\n",file=f)

except:
  print("\nNO IPC SENTENCE AND DOC OBJECT\n")

f.close()

#----------------------------------------------------



#for ent in doc.ents:
#  print("ents: ", ent.text, " ", ent.label_)

#nlp.tokenizer.add_special_case('-', [{ORTH: '-', POS:ADP}])

matcher = Matcher(nlp.vocab)


#pattern 00,01,10,11 : referanslar için (ör: a chassis, the chassis, vehicle chassis)
pattern00 = [{'POS': 'DET', 'OP': '+'},{'POS':'NOUN'},{'LIKE_NUM': True,'IS_DIGIT': True}]
pattern01 = [{'POS':'NOUN'},{'POS':'NOUN'},{'LIKE_NUM': True,'IS_DIGIT': True}]
pattern02 = [{'POS':'ADJ'},{'POS':'NOUN'},{'LIKE_NUM': True,'IS_DIGIT': True}]
pattern03 = [{'LIKE_NUM': True,'IS_DIGIT': False}, {'POS':'NOUN'},{'LIKE_NUM': True,'IS_DIGIT': True}]
pattern10 = [{'POS': 'DET', 'OP': '+'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True,'IS_DIGIT': True},{'TEXT':')'}]
pattern11 = [{'POS':'NOUN'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True,'IS_DIGIT': True},{'TEXT':')'}]
pattern12 = [{'POS':'ADJ'},{'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True,'IS_DIGIT': True},{'TEXT':')'}]
pattern13 = [{'LIKE_NUM': True,'IS_DIGIT': False}, {'POS':'NOUN'},{'TEXT':'('},{'LIKE_NUM': True,'IS_DIGIT': True},{'TEXT':')'}]

#pattern 2,3,4(?) : noun ve noun_phrase için sonrasında frekans belirlenerek SEARCH query keyword belirlemede kullanılıyor

pattern2 = [{'LIKE_NUM': True,'IS_DIGIT': False}, {'POS':'NOUN'}]
pattern3 = [{'POS':'NOUN'},{'POS':'NOUN'}]
#pattern4 = [{'POS':'ADJ'},{'POS':'NOUN'}]

matcher.add('REFERENCE',None,pattern00,pattern01,pattern02,pattern03,pattern10,pattern11,pattern12,pattern13)
found_matches = matcher(doc)

f = open('/content/drive/My Drive/MYOUTPUT.txt', 'a')

print("\nPOSSIBLE REFERENCES : \n",file=f)
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text,file=f)

print('\n',file=f)


if len(found_matches)==0:
  print("NO POSSIBLE REFERENCE found with given format\n",file=f)
  print("NO POSSIBLE REFERENCE found with given format\n")


referenceList = [doc[start:end].text for match_id, start, end in found_matches]
referenceList_freq = Counter(referenceList)
common_words = referenceList_freq.most_common(40)
print("\nPOSSIBLE REFERENCES (reference_phrase,freq): ", common_words,"\n", file=f)

print("SORTED POSSIBLE REFERENCES REGARDING FREQUENCY IN DESCENDING ORDER: \n",file=f)
print("SORTED POSSIBLE REFERENCES REGARDING FREQUENCY IN DESCENDING ORDER: \n")


sorted_reference_list = [word for (word, freq) in common_words]
for i in range(0,len(sorted_reference_list)):
  print(str(i+1)+")-  "+sorted_reference_list[i],file=f)
  print(str(i+1)+")-  "+sorted_reference_list[i])





matcher.remove('REFERENCE')
matcher.add('NOUN_PHRASE',None,pattern2,pattern3)
found_matches = matcher(doc)
print("\nPOSSIBLE NOUN PHRASES:\n", file=f)
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text,file=f)
    
    
startDocText = [doc[start:end].text for match_id, start, end in found_matches]


unexpected_element=[]
try:
  for i in range(0,len(startDocText)):
    if "-" in startDocText[i]:
      #print("unexpected element = ")
      unexpected_element.append(startDocText[i])
except IndexError:
  print("ERROR:index out of range",file=f)

print("List for unexpected element:",unexpected_element,file=f)

startDocText = [x for x in startDocText if x not in unexpected_element]


print("\nCRITICAL IPC KEYWORDS (comprising system,arrangement,product,device,method etc.):\n",file=f)
print("\nCRITICAL IPC KEYWORDS (comprising system,arrangement,product,device,method etc.):\n")


try:
    
  for i in range(len(startDocText)):
    if (startDocText[i].split()[1].lower()== "system" or
        startDocText[i].split()[1].lower()=="arrangement" or
        startDocText[i].split()[1].lower()=="arangement" or startDocText[i].split()[1].lower()=="apparatus" or
        startDocText[i].split()[1].lower()=="aparatus" or startDocText[i].split()[1].lower()=="device" or
        startDocText[i].split()[1].lower()=="devices" or
        startDocText[i].split()[1].lower()=="method" or startDocText[i].split()[1].lower()=="product" or 
        startDocText[i].split()[1].lower()=="production" or startDocText[i].split()[1].lower()=="assembly" or
        startDocText[i].split()[1].lower()=="asembly") :
      print(startDocText[i],file=f)
      print(startDocText[i])
      

except: 
  print("AN EXCEPTION OCCURED. UNEXPECTED CHARACTER-INDEX ERROR\n",file=f)

  
startDocText_freq = Counter(startDocText)
common_words = startDocText_freq.most_common(40)
print("\nIPC KEYWORDS (noun_phrase,freq): ", common_words,"\n",file=f)





  
#((vehicle seat)) ((folding mechanism)) (lock) before:publication:20140624

#https://patents.google.com/?q=(vehicle+seat)&q=(folding+mechanism)&q=lock&before=publication:20140624
#*************** common_words liste lengthi 2'den küçükse "list index out of range" hatası veriyor. Dolayısıyla for ile len(list) şeklinde sınırlandırmak doğru olabilir
print("\nSORTED IPC KEYWORDS QUERY FORMAT REGARDING FREQUENCY IN DESCENDING ORDER:\n",file=f)
print("\nSORTED IPC KEYWORDS QUERY FORMAT REGARDING FREQUENCY IN DESCENDING ORDER:\n")


query_words=[]
try:
  for i in range(0,len(common_words)):
    #print(startDocText[i][0],startDocText[i][-1])
    #if (startDocText[i][0] != '-' or startDocText[i][-1] != '-'):
    phrase_firstword = common_words[i][0].split()[0].lower()
    phrase_secondword = common_words[i][0].split()[1].lower()
    if (phrase_secondword[-1]=="s"):
      phrase_secondword = common_words[i][0].split()[1][:-1].lower()
    query_word = phrase_firstword+"_"+phrase_secondword+"+"
    query_words.append(query_word)
    print(str(i+1)+")-  "+query_word,file=f)
    print(str(i+1)+")-  "+query_word)

    
except:
  print('ERROR - UNEXPECTED CHARACTER-INDEX ERROR\n',file=f)


unique_keyword_list = [word for (word, freq) in common_words if freq is 1]
print("\nUNIQUE KEYWORDS (freq=1): \n",file=f)
print("\nUNIQUE KEYWORDS (freq=1): \n")

if (len(unique_keyword_list) ==0):
  print("NO UNIQUE KEYWORDS FOUND",file=f)
  print("NO UNIQUE KEYWORDS FOUND")

else:
  for i in range(len(unique_keyword_list)):
    print(unique_keyword_list[i],file=f)
    print(unique_keyword_list[i])




 
  
print("\nPROBABLE PATENW QUERY FOR IPC OR SIMILAR DOCUMENTS : \n",file=f)
print("\nPROBABLE PATENW QUERY FOR IPC OR SIMILAR DOCUMENTS : \n")

try:
  patenw_query = " "
  if (len(query_words) > 3):
    for i in range(0,3):
      patenw_query += query_words[i]+","
    print("and"+patenw_query[:-1],file=f)
    print("and"+patenw_query[:-1])
    url = "https://patents.google.com/?q=({})&q=({})&q=({})&before=publication:{}".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1],query_words[2].replace('_','+')[:-1],oldest_date)+"&clustered=true"
    print("\nFREQUENCY SEARCH URL: ",url)
    print("\nFREQUENCY SEARCH URL: ",url,file=f)
    compound_noun_url = "https://patents.google.com/?q=({})&q=({})&q=({})".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1],query_words[2].replace('_','+')[:-1])+ipc_url_noun_addition_string.replace('?','&')+"&before=publication:"+oldest_date+"&clustered=true"
    print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url)   
    print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url,file=f)  
    compound_chunk_url = "https://patents.google.com/?q=({})&q=({})&q=({})".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1],query_words[2].replace('_','+')[:-1])+ipc_url_chunk_addition_string.replace('?','&')+"&before=publication:"+oldest_date+"&clustered=true"
    print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url)   
    print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url,file=f)  

#ipc_search_url = "https://patents.google.com/"+ipc_url_addition_string+"&before=publication:"+oldest_date+"&clustered=true"


  elif (len(query_words) > 0):
    for i in range(0,len(query_words)):
      patenw_query += query_words[i]+","
    print("and"+patenw_query[:-1],file=f)
    print("and"+patenw_query[:-1])
    if (len(query_words) == 1):
      url = "https://patents.google.com/?q=({})&before=publication:{}".format(query_words[0].replace('_','+')[:-1],oldest_date)+"&clustered=true"
      print("\nFREQUENCY SEARCH URL: ",url)
      print("\nFREQUENCY SEARCH URL: ",url,file=f)
      compound_noun_url = "https://patents.google.com/?q=({})&q=({})&q=({})".format(query_words[0].replace('_','+')[:-1])+ipc_url_noun_addition_string+"&before=publication:"+oldest_date+"&clustered=true"
      print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url)   
      print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url,file=f)   
      compound_chunk_url = "https://patents.google.com/?q=({})&q=({})&q=({})".format(query_words[0].replace('_','+')[:-1])+ipc_url_chunk_addition_string+"&before=publication:"+oldest_date+"&clustered=true"
      print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url)   
      print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url,file=f)  

    elif (len(query_words) == 2):
      url = "https://patents.google.com/?q=({})&q=({})&before=publication:{}".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1],oldest_date)+"&clustered=true"
      print("\nFREQUENCY SEARCH URL: ",url)
      print("\nFREQUENCY SEARCH URL: ",url,file=f)
      compound_noun_url = "https://patents.google.com/?q=({})&q=({})".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1])+ipc_url_noun_addition_string+"&before=publication:"+oldest_date+"&clustered=true"
      print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url)   
      print("\nCOMPOUND NOUN SEARCH URL: ",compound_noun_url,file=f)     
      compound_chunk_url = "https://patents.google.com/?q=({})&q=({})".format(query_words[0].replace('_','+')[:-1],query_words[1].replace('_','+')[:-1])+ipc_url_chunk_addition_string+"&before=publication:"+oldest_date+"&clustered=true"
      print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url)   
      print("\nCOMPOUND CHUNK SEARCH URL: ",compound_chunk_url,file=f)    


  else: 
    print("""NO NOUN PHRASE FOUND... NO RESULTS FOR QUERY !""",file=f)
    print("\nNO FREQUENCY SEARCH URL\n",file=f)
    print("\nNO COMPOUND SEARCH URL ONLY IPC SEARCH URL PRODUCED\n",file=f)

    print("""NO NOUN PHRASE FOUND... NO RESULTS FOR QUERY !""")
    print("\nNO FREQUENCY SEARCH URL\n")
    print("\nNO COMPOUND SEARCH URL ONLY IPC SEARCH URL PRODUCED\n")

  print("\nIPC NOUN SEARCH URL: ",ipc_noun_search_url,file=f)
  print("\nIPC NOUN SEARCH URL: ",ipc_noun_search_url)
  print("\nIPC CHUNK SEARCH URL: ",ipc_chunk_search_url,file=f)
  print("\nIPC CHUNK SEARCH URL: ",ipc_chunk_search_url)

except:
  print("\nNO COMPOUND AND IPC URL ONLY FREQUENCY URL EXISTS\n")

if choice_text.lower()=="d":
  os.rename('/content/drive/My Drive/MYOUTPUT.txt','/content/drive/My Drive/MYOUTPUT-D.txt')
 
else: 
  os.rename('/content/drive/My Drive/MYOUTPUT.txt','/content/drive/My Drive/MYOUTPUT-C.txt') 

  
f.close()


In [0]:
#GOOGLE DOCUMENT LIST MODULE******************************
import pandas as pd
import matplotlib.pyplot as plt 
# url= 'https://patents.google.com/?q=%28base+lock%29&q=%28lock+folding%29&q=%28folding+mechanism%29&q=%28base+lock+mechanism%29&q=%28seat+lock+folding+mechanism%29&q=%28vehicle+rear+seat%29&q=B60N2%252f00&q=weight+housing&q=rotation+bracket&q=pivoting+bracket&q=bolt&q=bearing&before=publication:20140627&clustered=true'
df = pd.read_csv('/content/drive/My Drive/gps.csv')

# df.index  --> ('DE-202009002580-U1', ...), alt alta hep böyle

#df[url].index[0][0] # 'id'
#df[url].index[1][0] #'CN-103380025-A'
#df[url].index[2][0] #'WO-9911488-A1'
#df[url].index[3][0] #'DE-10127067-A1'

range_documents=int(input("Enter number of documents+1: "))
for i in range(1,range_documents):
  print(str(df['search URL:'].index[i][0]).replace('-','')) 

 



In [0]:
# SIMILAR WORDS MODULE************************************
from scipy import spatial

cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)

wordforSimilarity= input("Insert a word for similar matches: ")
wordforSimilarity = wordforSimilarity.lower()
new_vector = nlp.vocab[wordforSimilarity].vector

computed_similarities = []

for word in nlp.vocab:
    # Ignore words without vectors and mixed-case words:
    if word.vector_norm > 0:
        if word.is_lower:
          if word.is_alpha:
            similarity = cosine_similarity(new_vector, word.vector)
            computed_similarities.append((word, similarity))

computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])

print([w[0].text for w in computed_similarities[:20]])


In [0]:
#IPC-PREDICT-NOUN PHRASE
#2 AŞAMALI- 3 HANELİ IPC

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial

nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()


input_doc = nlp(my_input)

#for ent in doc.ents:
#  print("ents: ", ent.text, " ", ent.label_)

#nlp.tokenizer.add_special_case('-', [{ORTH: '-', POS:ADP}])

matcher = Matcher(nlp.vocab)
#pattern 00,01,10,11 : referanslar için (ör: a chassis, the chassis, vehicle chassis)
patternipc1 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc2 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc3 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'ADJ'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc4 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'ADJ'},{'POS':'NOUN'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
patternipc5 = [{'POS': 'NOUN', 'OP': '!'},{'POS':'NOUN'},{'POS': 'NOUN', 'OP': '!'}]
matcher.add('IPC',None,patternipc1,patternipc2,patternipc3,patternipc4,patternipc5)
found_matches = matcher(input_doc)

print('\nPOSSIBLEIPC WORDS:\n')
if len(found_matches)==0:
  print("NO POSSIBLE IPC WORD found with given format\n")

list_ipcwords = []
for match_id, start, end in found_matches:
  string_id = nlp.vocab.strings[match_id]  # get string representation
  span = input_doc[start:end]                    # get the matched span
  print(match_id, string_id, start, end, span.text)
  list_ipcwords.append(span.text)
print('\n')

ipctext = ' '.join(list_ipcwords)
rawdoc = nlp(ipctext)
apptext=[]
for token in rawdoc:
  if token.pos_ == 'NOUN' or token.pos_ == 'ADJ':
    apptext.append(token.text)
# unique ifadeler bulunup listeye dahil edilir, listeden tek bir string(unique_text) üretilir,bu string de nlp ile doc objesine dönüştürülür 
unique_list = [] 
for x in apptext:
  if x not in unique_list: 
    unique_list.append(x) 

unique_text = ' '.join(unique_list)
appdoc = nlp(unique_text)
print("\nAPPDOC: ",appdoc)


df = pd.read_csv('/content/drive/My Drive/ipc.tsv', sep='\t')

# ilk cümle başarısız olursa alttaki 2 yorum koduna devam edilebilir
#app = input("insert critical words for ipc: ")
#appdoc= nlp(app)
probIpc=[]
for i in(range(len(df['Definition']))):
  print(appdoc.similarity(nlp(str(df['Definition'][i]))))
  probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))

print("FIRST FIVE MAXIMUM VALUES:\n")

df['predictionValue']=pd.Series(probIpc)
print(df.sort_values(by='predictionValue', ascending = False))
print("\n")

try:
  print("VALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
except:
  print('NO PREDICTION VALUE GREATER THAN 0.5\n')


answer=input("\nIf you are not satisfied with the results do you want to insert your own noun phrases for ipc ?\n")

if answer.lower() == "y" or answer.lower() == "yes":
  app = input("Insert critical noun phrases for ipc wihout punctuation (e.g image processor, marine, communication network ): ")
  appdoc= nlp(app)
  probIpc=[]
  for i in(range(len(df['Definition']))):
    print(appdoc.similarity(nlp(str(df['Definition'][i]))))
    probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))
  print("\nMAXIMUM VALUES:\n")
  df['predictionValue']=pd.Series(probIpc)
  print(df.sort_values(by='predictionValue', ascending = False).head(20))
  print("\n")
  
  try:
    print("VALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
    print("\nYou can run again for new search...")
  except:
    print('NO PREDICTION VALUE GREATER THAN 0.5\n')

else:
  print("\nYou can run again for new search...")





In [0]:
# SIMILARITY EXERCISE
app=input("app: ")
ipc=input("ipc: ")
docipc = nlp(ipc)
docapp = nlp(app)
print(docapp.vector_norm,docipc.vector_norm)
print("app:",docapp.has_vector)
print("ipc:",docipc.has_vector)
print(docapp.similarity(docipc))

In [0]:
# TOKEN ATTRIBUTES EXERCISE
entry=input("Bir cümle yaz: ")
doc=nlp(entry.lower())
for chunk in doc.noun_chunks:
  liste=[chunk.text for chunk in doc.noun_chunks]
print("liste: ",liste)
for token in doc:
  print(token.text,token.lemma_,token.pos_,token.tag_,token.dep_,token.shape_,token.is_alpha,token.is_stop,token.like_num)

In [0]:
#IPC PREDICT-NOUN-CHUNKS (+ (TOKEN.TAG_=VBG) ?)
# 2 AŞAMALI
import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial

nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()

#--------
input_doc = nlp(my_input)
liste_chunk=[]
for chunk in input_doc.noun_chunks:
  print("CHUNK: ",chunk)
  for i in(range(len(chunk))):
    if (chunk[i].is_stop == False and chunk[i].text != "-" and chunk[i].is_punct == False and chunk[i].pos_ != 'NUM'):
      liste_chunk.append(chunk[i].text)
#---------

print("CHUNK LİSTESİ: ",liste_chunk)

ipctext = ' '.join(liste_chunk)


appdoc = nlp(ipctext)
print("\nAPPDOC: ",appdoc)


df = pd.read_csv('/content/drive/My Drive/ipc.tsv', sep='\t')


probIpc=[]
for i in(range(len(df['Definition']))):
  print(appdoc.similarity(nlp(str(df['Definition'][i]))))
  probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))

print("\nMAXIMUM VALUES: \n")

df['predictionValue']=pd.Series(probIpc)
print(df.sort_values(["predictionValue"], ascending = False).head(20))
print("\n")

#burası çok önemli yeni df'nin indexi sıfırlanmalı
df2 = df.sort_values(["predictionValue"], ascending = False)
df2 = df2.reset_index(drop=True)

#most prop ipc list, df2'deki IPC verileri ile oluşturulur
most_prob_ipc_list=[]
for i in range(0,20):
  most_prob_ipc_list.append((df2['IPC'][i],i))

final_ipc_df1 = pd.DataFrame(most_prob_ipc_list, columns =['IPC','FIRST-INDEX'])
#print("MOST PROBABLE IPC LIST: \n",most_prob_ipc_list)



try:
  print("\nVALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
except:
  print('NO PREDICTION VALUE GREATER THAN 0.5\n')


answer=input("\nIf you are not happy with the results do you want to insert your own noun phrases for ipc ?\n")

if answer.lower() == "y" or answer.lower() == "yes":
  app = input("Insert critical noun phrases for ipc wihout punctuation (e.g image processor, marine, communication network ): ")
  appdoc= nlp(app)
  probIpc=[]
  for i in(range(len(df['Definition']))):
    print(appdoc.similarity(nlp(str(df['Definition'][i]))))
    probIpc.append(appdoc.similarity(nlp(str(df['Definition'][i]))))
  print("\nMAXIMUM VALUES:\n")
  df['predictionValue']=pd.Series(probIpc)
  print(df.sort_values(["predictionValue"], ascending = False).head(20))

  print("\nFIRST PREDICTION VALUES:\n", df2.head(20))
  
  # ilk df2'dn farklı olarak ikinci df2 oluşturulur sonuçta her iki df2'deki IPC verileri ile most prob ipc list oluşur
  df2=df.sort_values(["predictionValue"], ascending = False)
  df2 = df2.reset_index(drop=True)

  most_prob_ipc_list2=[]
  for m in range(0,20):
    most_prob_ipc_list2.append((df2['IPC'][m],m))
  #print("MOST PROBABLE IPC LIST: \n",most_prob_ipc_list)
  final_ipc_df2 = pd.DataFrame(most_prob_ipc_list2, columns =['IPC','SECOND-INDEX'])
  result_df = pd.merge(final_ipc_df1,final_ipc_df2, on='IPC', how='outer', indicator=True)

  #result_df = pd.merge(final_ipc_df1,final_ipc_df2, on='IPC')
  #result_df = final_ipc_df1.append(final_ipc_df2, sort=False)
  print("\nRESULT:\n",result_df)
  #final_ipc_list = [(x, most_prob_ipc_list.count(x),most_prob_ipc_list.) for x in set(most_prob_ipc_list)]
  #print ("IPC FREQUENCY IN BOTH METHODS: ", final_ipc_list) 
  
  #final_ipc_df = pd.DataFrame(final_ipc_list, columns =['IPC', 'FREQUENCY', 'FIRST-INDEX']) 
  #-----alttaki komut önemli, son sorting
  result_df['TOTAL-INDEX'] = result_df['FIRST-INDEX']+result_df['SECOND-INDEX']

  final_sorted_ipc_df = result_df.sort_values(["TOTAL-INDEX"], ascending = True)
  print("\nIPC FREQUENCY IN BOTH METHODS:\n ",final_sorted_ipc_df)

  try:
    print("\nVALUES GREATER THAN 0.5:\n",df[df['predictionValue'] > 0.5])
    print("\nYou can run again for new search...")
  except:
    print('NO PREDICTION VALUE GREATER THAN 0.5\n')

else:
  print("\nYou can run again for new search...")





In [0]:
#IPC PREDICT-NOUN-CHUNKS-FULL IPC- 2 KUTUCUKLU DENEME

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial
from collections import Counter


nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input = my_input.lower()

#--------
input_doc = nlp(my_input)


liste_chunk=[]
for chunk in input_doc.noun_chunks:
  print("CHUNK: ",chunk)
  for i in(range(len(chunk))):
    if (chunk[i].is_stop == False and chunk[i].text != "-" and chunk[i].is_punct == False and chunk[i].pos_ != 'NUM'):
      liste_chunk.append(chunk[i].text)
#---------


print("CHUNK LİSTESİ: ",liste_chunk)

most_freq_tuple_list = Counter(liste_chunk).most_common(10)
most_freq_noun_list = [ word for word,freq in most_freq_tuple_list]
print("MOST FREQ NOUNS: ",most_freq_noun_list)

#---------------------------------------------
#burada en yüksek frekanslı kelimelerle bir tex oluşturulur ve sonrasında bu doc objesine dönüştürülür
ipctext = ' '.join(most_freq_noun_list)

appdoc = nlp(ipctext)
print("\nAPPDOC: ",appdoc)
#--------------------------------------------

df = pd.read_csv('/content/drive/My Drive/FULLIPCVEHICLE.tsv', sep='\t')

#BURADA DEF1 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC1 LİSTESİNE EKLENİR. DEF1 4 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc1=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF1'][i]))))
  probIpc1.append(appdoc.similarity(nlp(str(df['DEF1'][i]))))

print("\nMAXIMUM VALUES FOR DEF1: \n")

df['predictionValue1']=pd.Series(probIpc1)
print(df.sort_values(["predictionValue1"], ascending = False).head(20))
print("\n")

#BURADA DEF2 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC2 LİSTESİNE EKLENİR. DEF2 3 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc2=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF2'][i]))))
  probIpc2.append(appdoc.similarity(nlp(str(df['DEF2'][i]))))

print("\nMAXIMUM VALUES FOR DEF2: \n")

df['predictionValue2']=pd.Series(probIpc2)
print(df.sort_values(["predictionValue2"], ascending = False).head(20))
print("\n")

#burası çok önemli yeni df'nin indexi sıfırlanmalı
df['FULLIPC-Prediction']=df['predictionValue1']+df['predictionValue2']
df2 = df.sort_values(["FULLIPC-Prediction"], ascending = False)
df2 = df2.reset_index(drop=True)
print("FULL IPC PREDICTION MAX. VALUES:\n",df2)

# GP-search terms example((side airbag)) ((trim piece)) ((seatback)) before:publication:20140402


In [0]:
#IPC PREDICT-TOKENS-FULL IPC - 2 KUTUCUKLU DENEME - 3 KUTUCUK ?

import pandas as pd
import spacy
from spacy.matcher import Matcher
from scipy import spatial
from collections import Counter


nlp = spacy.load('en_core_web_lg')

my_input = input("Insert the first meaningful sentence in description regarding ipc: ")
my_input=my_input.lower()

#--------
input_doc = nlp(my_input)

#--------------------
#burada token listesi isim ve 'ing' ile biten isimfiilden oluşacak şekilde üretilir

token_list=[]
for token in input_doc:
  if token.pos_== 'NOUN' or token.tag_=='VBG':
    if not token.text in ['invention','product','method','system','prior','mechanism','apparatus','device','embodiment','art','state']: 
      token_list.append(token.text)

most_freq_tuple_list = Counter(token_list).most_common(10)
print("FREKANSLI LİSTE: \n",most_freq_tuple_list)
most_freq_noun_list = [ word for word,freq in most_freq_tuple_list]
print("MOST FREQ NOUNS: ", most_freq_noun_list)
#--------------------

#---------------------------------------------
#burada en yüksek frekanslı kelimelerle bir tex oluşturulur ve sonrasında bu doc objesine dönüştürülür
ipctext = ' '.join(most_freq_noun_list)

appdoc = nlp(ipctext)
print("\nAPPDOC: ",appdoc)
#---------------------------------------------

df = pd.read_csv('/content/drive/My Drive/FULLIPCVEHICLE.tsv', sep='\t')

#BURADA DEF1 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC1 LİSTESİNE EKLENİR. DEF1 4 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc1=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF1'][i]))))
  probIpc1.append(appdoc.similarity(nlp(str(df['DEF1'][i]))))

print("\nMAXIMUM VALUES FOR DEF1: \n")

df['predictionValue1']=pd.Series(probIpc1)
print(df.sort_values(["predictionValue1"], ascending = False).head(20))
print("\n")

#BURADA DEF2 SÜTUNU İLE BENZERLİK DEĞERLERİ PROBIPC2 LİSTESİNE EKLENİR. DEF2 3 HANELİ IPC SINIFI İÇİN TANIMDIR
probIpc2=[]
for i in(range(len(df['IPC']))):
  #print(appdoc.similarity(nlp(str(df['DEF2'][i]))))
  probIpc2.append(appdoc.similarity(nlp(str(df['DEF2'][i]))))

print("\nMAXIMUM VALUES FOR DEF2: \n")

df['predictionValue2']=pd.Series(probIpc2)
print(df.sort_values(["predictionValue2"], ascending = False).head(20))
print("\n")

#burası çok önemli yeni df'nin indexi sıfırlanmalı
df['FULLIPC-Prediction']=df['predictionValue1']+df['predictionValue2']
df2 = df.sort_values(["FULLIPC-Prediction"], ascending = False)
df2 = df2.reset_index(drop=True)
print("FULL IPC PREDICTION MAX. VALUES:\n",df2)

# GP-search terms example:((side airbag)) ((trim piece)) ((seatback)) before:publication:20140402
